In [1]:
from glob import glob
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image, ImageDraw
import os
from datasets import basic_entities as entities
import pickle
from math import floor

In [4]:
# Format (point1, point2, ped_size(head-torso) (x,y)
lines = {
    'IM01': [
        # Bottom
        ((150, 700), (750, 450), (35, 75)),
        # Top
        ((60, 220), (500, 50), (55, 50))
    ],
    'IM02': [
        # Bottom left
        ((320, 400), (500, 450), (45,55)),
        # Bottom right
        ((550, 400), (900, 470), (35,55)),
        # Middle
        ((400, 200), (850, 260), (30,45)),
        # Top
        ((450, 50), (900, 100), (20,40))
    ],
    'IM03': [
        # Middle all
        ((600, 550), (640, 250), (20, 20)),
        # Single line, top
        ((800, 380), (810, 300), (20, 20)),
        # Small line, right
        ((1000, 390), (980, 600), (20, 25))
    ],
    'IM04': [
        # Top centre
        ((740, 160), (830, 90), (30,20)),
        # Left centre
        ((750, 350), (920, 480), (30,30)),
        # Top
        ((200, 200), (520, 0), (30, 30)),
        # Left
        ((20, 250), (300, 720), (25,25))
    ],
    'IM05': [
        # Bottom
        ((600, 400), (610, 719), (35,45)),
        # Right
        #((800, 300), (1200, 250), (20,35)),
        # Left
        ((0, 250), (350, 380), (45,40))
    ]
}

# Get per video when a pedestrian crosses a line (Get all the information of all the lines of the video)
def get_line_crossing_frames(video):
    # So this function returns a tupple with each an array which gives the frame number that a person crossed a line.
    # This makes it pretty easy to split the long videos into smaller samples
    base = os.path.basename(video.get_path())

    track_path = '{}/PersonTracks.pb'.format(video.get_path().replace('images', 'gt_trajectories'))
    ret = pickle.load(open(track_path, 'rb'))

    video_crosses = {}
    for o, line in enumerate(lines[base]):

        crosses = [[], []]
        for i, pedestrian in enumerate(ret['GT_Trajectories']):

            arr = np.array(pedestrian)
            arr = arr[:, [1, 0]]

            base_point = np.array(line[0])
            vector = np.array(line[1])
            norm_vector = vector - base_point
            norm_arr = arr - base_point

            # Project to check which points fall inside the line
            upper_proj = np.dot(norm_arr, norm_vector)
            lower_proj = np.linalg.norm(norm_vector) ** 2
            proj = upper_proj / lower_proj
            inside = np.array([proj >= 0, proj <= 1]).all(axis=0)

            # Are you on the right side of the line
            check = (norm_vector[0] * norm_arr[:, 1] - norm_vector[1] * norm_arr[:, 0]) > 0;

            # If it didn't cross, then no need to check further
            if np.all(check == check[0]):
                continue

            check2 = np.roll(check, 1)
            together = check[1:] != check2[1:]
            poses = np.where(together == True)[0]

            pos = None
            for pos_i in poses:
                if inside[pos_i] == False:
                    continue

                # Some weird start displacements are filtered out this way
                norm = np.linalg.norm([arr[pos_i, 0] - arr[pos_i + 1, 0], arr[pos_i, 1] - arr[pos_i + 1, 1]])
                if norm > 20:
                    continue

                pos = pos_i

            if pos == None:
                continue

            crossing_frame = int(ret['GT_StartPoints'][i][0] + pos + 2)
            crosses[int(check[pos + 1])].append(crossing_frame)

        video_crosses[o] = crosses

    return video_crosses


def load_all_videos(path, load_peds=True):
    # Load all the videos from the path and load it into vidoe/frame objects
    videos = []
    for image_dir in glob('{}/images/IM??'.format(path)):
        base = os.path.basename(image_dir)
        frame_files = glob('{}/*.png'.format(image_dir))
        frame_files.sort()

        video = entities.BasicVideo(image_dir)
        videos.append(video)
        for file_path in frame_files:
            frame_obj = entities.BasicFrame(file_path)
            video.add_frame(frame_obj)

        if base not in lines:
            print("Skipped {}".format(base))
            continue

        frames = video.get_frames()

        # Open pedestrians information
        if load_peds:
            track_path = '{}/PersonTracks.pb'.format(image_dir.replace('images', 'gt_trajectories'))
            ret = pickle.load(open(track_path, 'rb'))

            # Loop per pedestrian over the frames
            for i, pedestrian in enumerate(ret['GT_Trajectories']):
                arr = np.array(pedestrian)
                arr = arr[:, [1, 0]]

                for o, loc in enumerate(arr):
                    frame_id = int(ret['GT_StartPoints'][i][0] + o)
                    frames[frame_id].add_point((loc[0], loc[1]))
                    # TODO: Add tracking, now we haven't connected the track between frames.


            # Load crosses into lines
            crosses = get_line_crossing_frames(video)
            video_lines = lines[os.path.basename(video.get_path())]
            for o, line in enumerate(video_lines):
                line_obj = entities.BasicLineSample(video, line[0], line[1])
                video.add_line(line_obj)

                for cross in crosses[o][0]:
                    line_obj.add_crossing(cross, 0)

                for cross in crosses[o][1]:
                    line_obj.add_crossing(cross, 1)



    return videos

In [5]:
load_all_videos('data/TUBCrowdFlow')